In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np


from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import pickle
from tensorflow import keras

# Data and Model prep

In [2]:
data_directory = "data/HMDA/"

X_test = pd.read_csv(data_directory+'HMDA-MORTGAGE-APPROVAL_Xtest.bz2')
y_test = pd.read_csv(data_directory+'HMDA-MORTGAGE-APPROVAL_ytest.bz2')
X_train = pd.read_csv(data_directory+'HMDA-MORTGAGE-APPROVAL_Xtrain.bz2')
y_train = pd.read_csv(data_directory+'HMDA-MORTGAGE-APPROVAL_ytrain.bz2')

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(650877, 35)
(650877, 1)
(278948, 35)
(278948, 1)


In [3]:
def load_model(model_type):

    if model_type == 'dt':
        model = pickle.load(open(data_directory+'HMDA-MORTGAGE-APPROVAL_DT_scaling_20210205_014819.pkl', 'rb'))
    elif model_type == 'gbc':
        model = pickle.load(open(data_directory+'HMDA-MORTGAGE-APPROVAL_GBC_scaling_20210205_014418.pkl', 'rb'))
    elif model_type == 'lr':
        model = pickle.load(open(data_directory+'HMDA-MORTGAGE-APPROVAL_LR_scaling_20210205_012956.pkl', 'rb'))
    elif model_type == 'rf':
        model =  pickle.load(open(data_directory+'HMDA-MORTGAGE-APPROVAL_RF_scaling_20210205_013239.pkl', 'rb'))
    else:
        model = keras.models.load_model(data_directory+'HMDA-MORTGAGE-APPROVAL_MLP_scaling_20210205_011811.h5')
        
    return model

# Attack - Lookup

In [4]:
from uret.utils.config import process_config_file
import random

cf = "configs/HMDA/random.yml"
num_samples = 1000
scaler = StandardScaler().fit(X_train)

def feature_extractor(x):
    if len(np.shape(x)) == 2:
        return np.array(scaler.transform(x))
    else:
        return np.array(scaler.transform([x]))

x_transformed = scaler.transform(X_test)

for mt in ['dt', 'gbc', 'lr', 'rf']:
    print("Model type:", mt)
    model = load_model(mt)
    
    model_preds = np.argmax(model.predict_proba(x_transformed),axis=1)
    
    zero_inds = np.where(model_preds == 0)[0][:num_samples]
    one_inds = np.where(model_preds == 1)[0][:num_samples]
    
    task_samples = np.concatenate((list(X_test.values[zero_inds]), list(X_test.values[one_inds])))
    
    orig_model_preds = np.argmax(model.predict_proba(feature_extractor(task_samples)),axis=1)
    
    explorer = process_config_file(cf, model, feature_extractor=feature_extractor, input_processor_list=[])
    
    adv_samples = explorer.explore(task_samples)  
    
    adv_model_preds = np.argmax(model.predict_proba(feature_extractor(adv_samples)),axis=1)
    
    success_rate = np.sum(adv_model_preds != orig_model_preds)/len(orig_model_preds)
    
    print("Success rate:", success_rate)
    print()
    
    pickle.dump([task_samples, adv_samples], open('data/HMDA/random_adv_samples_' + mt +'.p', 'wb'))

Model type: dt


100%|█████████████████████████████████████| 2000/2000 [00:01<00:00, 1741.56it/s]


Success rate: 0.389

Model type: gbc


100%|██████████████████████████████████████| 2000/2000 [00:02<00:00, 752.71it/s]


Success rate: 0.141

Model type: lr


100%|█████████████████████████████████████| 2000/2000 [00:01<00:00, 1559.69it/s]


Success rate: 0.37

Model type: rf


100%|███████████████████████████████████████| 2000/2000 [06:26<00:00,  5.18it/s]


Success rate: 0.26



In [5]:
cf = "configs/HMDA/random_mlp.yml"
for mt in ['mlp']:
    print("Model type:", mt)
    model = load_model(mt)
    
    model_preds = np.argmax(model.predict(x_transformed),axis=1)
    
    zero_inds = np.where(model_preds == 0)[0][:num_samples]
    one_inds = np.where(model_preds == 1)[0][:num_samples]
    
    task_samples = np.concatenate((list(X_test.values[zero_inds]), list(X_test.values[one_inds])))
    
    orig_model_preds = np.argmax(model.predict(feature_extractor(task_samples)),axis=1)
    
    explorer = process_config_file(cf, model, feature_extractor=feature_extractor, input_processor_list=[])
    
    adv_samples = explorer.explore(task_samples)  
    
    adv_model_preds = np.argmax(model.predict(feature_extractor(adv_samples)),axis=1)
    
    success_rate = np.sum(adv_model_preds != orig_model_preds)/len(orig_model_preds)
    
    print("Success rate:", success_rate)
    print()
    
    pickle.dump([task_samples, adv_samples], open('data/HMDA/random_adv_samples_' + mt +'.p', 'wb'))

2022-10-18 09:07:32.314526: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model type: mlp
63/63 [==============================] - 0s 400us/step


  0%|                                                  | 0/2000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 10ms/step


  0%|                                          | 2/2000 [00:00<04:00,  8.32it/s]

...

1/1 [==============================] - 0s 11ms/step


100%|██████████████████████████████████████▉| 1999/2000 [04:49<00:00,  7.01it/s]

1/1 [==============================] - 0s 10ms/step


100%|███████████████████████████████████████| 2000/2000 [04:49<00:00,  6.91it/s]

63/63 [==============================] - 0s 408us/step
Success rate: 0.365

